In [1]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
traindata = 'D:\\Study\\1st Semester\\Machine Learning\\Assignments\\Assignment2\\train'
validationdata = 'D:\\Study\\1st Semester\\Machine Learning\\Assignments\\Assignment2\\validation'
testdata = 'D:\\Study\\1st Semester\\Machine Learning\\Assignments\\Assignment2\\test\\test'

In [4]:
# Define image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for validation data

train_generator = train_datagen.flow_from_directory(
    traindata,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    validationdata,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 20000 images belonging to 4 classes.
Found 4000 images belonging to 4 classes.


In [5]:
class_mapping = train_generator.class_indices
print(class_mapping)

{'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3}


In [6]:
from tensorflow.keras.applications import InceptionV3
input_shape = (224, 224, 3)
num_classes = 4
model_inception = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
for layer in model_inception.layers:
    layer.trainable = False
x = model_inception.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model_inception_modified = Model(inputs=model_inception.input, outputs=predictions)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)
model_inception_modified.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model_inception_modified.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=valid_generator,
    epochs=6,
    verbose=1
)

Epoch 1/6
625/625 [==============================] - 910s 1s/step - loss: 6.6185 - accuracy: 0.2736 - val_loss: 1.3865 - val_accuracy: 0.2500
Epoch 2/6
625/625 [==============================] - 696s 1s/step - loss: 1.3965 - accuracy: 0.2506 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 3/6
625/625 [==============================] - 712s 1s/step - loss: 1.3914 - accuracy: 0.2458 - val_loss: 1.3865 - val_accuracy: 0.2500
Epoch 4/6
625/625 [==============================] - 759s 1s/step - loss: 1.3881 - accuracy: 0.2507 - val_loss: 1.3871 - val_accuracy: 0.2500
Epoch 5/6
625/625 [==============================] - 659s 1s/step - loss: 1.3899 - accuracy: 0.2412 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 6/6
625/625 [==============================] - 672s 1s/step - loss: 1.3887 - accuracy: 0.2480 - val_loss: 1.3868 - val_accuracy: 0.2500


In [8]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense

# Load pre-trained DenseNet121 model without top layers (include_top=False)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)  # num_classes is the number of classes in your dataset

# Create model
model_densenet = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)
model_densenet.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])

29084464/29084464 [==============================] - 7s 0us/step


In [9]:
# Train the model
model_densenet.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // 32,
    epochs=10,
    verbose=1
)

Epoch 1/10
625/625 [==============================] - 816s 1s/step - loss: 1.2010 - accuracy: 0.4853 - val_loss: 0.8995 - val_accuracy: 0.6370
Epoch 2/10
625/625 [==============================] - 671s 1s/step - loss: 1.0473 - accuracy: 0.5264 - val_loss: 0.9535 - val_accuracy: 0.6148
Epoch 3/10
625/625 [==============================] - 692s 1s/step - loss: 1.0297 - accuracy: 0.5478 - val_loss: 0.8470 - val_accuracy: 0.6470
Epoch 4/10
625/625 [==============================] - 486s 778ms/step - loss: 1.0045 - accuracy: 0.5688 - val_loss: 0.8470 - val_accuracy: 0.6787
Epoch 5/10
625/625 [==============================] - 441s 705ms/step - loss: 1.0000 - accuracy: 0.5706 - val_loss: 0.8593 - val_accuracy: 0.6870
Epoch 6/10
625/625 [==============================] - 487s 780ms/step - loss: 0.9978 - accuracy: 0.5708 - val_loss: 0.8134 - val_accuracy: 0.6955
Epoch 7/10
625/625 [==============================] - 556s 889ms/step - loss: 0.9845 - accuracy: 0.5824 - val_loss: 0.8123 - val_accu

In [10]:
from tensorflow.keras.applications import MobileNetV2

# Load pre-trained MobileNetV2 model without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)  # num_classes is the number of classes in your dataset

# Create model
model_mobilenet = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)
model_mobilenet.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 [==============================] - 1s 0us/step


In [11]:
# Train the model
model_mobilenet.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=valid_generator,
    epochs=6,
    verbose=1
)

Epoch 1/6
625/625 [==============================] - 296s 465ms/step - loss: 1.1922 - accuracy: 0.5611 - val_loss: 0.7691 - val_accuracy: 0.7318
Epoch 2/6
625/625 [==============================] - 278s 445ms/step - loss: 0.9754 - accuracy: 0.6001 - val_loss: 0.7809 - val_accuracy: 0.7377
Epoch 3/6
625/625 [==============================] - 277s 443ms/step - loss: 0.9517 - accuracy: 0.6132 - val_loss: 0.8100 - val_accuracy: 0.7228
Epoch 4/6
625/625 [==============================] - 275s 440ms/step - loss: 0.9695 - accuracy: 0.6040 - val_loss: 0.8014 - val_accuracy: 0.7017
Epoch 5/6
625/625 [==============================] - 274s 438ms/step - loss: 0.9568 - accuracy: 0.6128 - val_loss: 0.7804 - val_accuracy: 0.7272
Epoch 6/6
625/625 [==============================] - 277s 443ms/step - loss: 0.9427 - accuracy: 0.6210 - val_loss: 0.7326 - val_accuracy: 0.7650


In [12]:
test_generator = train_datagen.flow_from_directory(
     'D:\\Study\\1st Semester\\Machine Learning\\Assignments\\Assignment2\\test',
     target_size=(256,256),
    batch_size=32,
    class_mode= None)

Found 10000 images belonging to 1 classes.


In [13]:
class_mapping= {v: k for k, v in class_mapping.items()}
class_mapping[0]='CNV'
class_mapping[1]='DME'
class_mapping[2]='DRUSEN'
class_mapping[3]='NORMAL'
print(class_mapping)

{'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3, 0: 'CNV', 1: 'DME', 2: 'DRUSEN', 3: 'NORMAL'}


In [ ]:
import numpy as np
import csv
predictions = model_mobilenet.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
predicted_class_names = [class_mapping[label] for label in predicted_labels]
with open('Rahul_12310890.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows([[label] for label in predicted_class_names])